<a href="https://colab.research.google.com/github/RiccardoRubini93/SWA_DQ/blob/main/Bucket_file_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
from io import BytesIO, StringIO
from google.cloud import storage

def get_byte_fileobj(project: str,
                     bucket: str,
                     path: str) -> BytesIO:
    """
    Retrieve data from a given blob on Google Storage and pass it as a file object.
    :param path: path within the bucket
    :param project: name of the project
    :param bucket_name: name of the bucket
    :return: file object (BytesIO)
    """
    blob = _get_blob(bucket, path, project)
    byte_stream = BytesIO()
    blob.download_to_file(byte_stream)
    byte_stream.seek(0)
    return byte_stream

def get_bytestring(project: str,
                   bucket: str,
                   path: str) -> bytes:
    """
    Retrieve data from a given blob on Google Storage and pass it as a byte-string.
    :param path: path within the bucket
    :param project: name of the project
    :param bucket_name: name of the bucket
    :return: byte-string (needs to be decoded)
    """
    blob = _get_blob(bucket, path, project)
    s = blob.download_as_string()
    return s


def _get_blob(bucket_name, path, project):

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    return blob

In [5]:
def check_for_string_in_blob(project, bucket_name, path,tgt_string):
          fileobj = get_byte_fileobj(project, bucket_name, path)
          blob = _get_blob(bucket_name, path, project)
          with blob.open("r") as f:
               #print(f.read())
               file_text=f.read()
          if tgt_string in file_text:
            #print(f'String found in {path}')
            return file_text.count(tgt_string)
          else:
            return 0

In [10]:
#define the bucket where to look for missing pspReference

project_id = 'aa-fraud-detection'
client = storage.Client(project_id)

bucket = 'data_exchange__adyen_prod' 
prefix = 'v1/2023'

count = 0
cnt = 0

for blob in client.list_blobs(bucket, prefix = prefix):
  cnt +=1
  path = blob.name
  tgt_string = '10GB00002865766'
  out = check_for_string_in_blob(project_id,bucket,path,tgt_string)
  
  if out > 0 : 
    count += 1
    print(f"{count} on {cnt} files have the target string in the text. Occurencies {out}. Blob name : {blob.name}")

KeyboardInterrupt: ignored